In [1]:
from wrangle import get_311_data, clean_311, split_separate_scale
from explore import get_chi_square, get_anova_test, make_dbad_relplot
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
df_1= get_311_data()
df = clean_311(df_1)

In [3]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test = split_separate_scale(df, stratify_by= None)

KeyError: "['zipcode'] not found in axis"

In [ ]:
df.info()

In [ ]:
df.council_district.isna().sum()

In [ ]:
train.head()

5. Does category/department affect response time?

In [ ]:
sns.distplot(train.days_before_or_after_due)

In [ ]:
train.columns

6. Is response time for issue X different between geographical location Y when compared to the average response time? (or other geographical location?)

In [ ]:
train.columns

#### Hypothesis Testing #1 (ANOVA)

$H_0$: There is no difference in days before or after due date between the districts.

$H_a$: There is a significant difference in days before or after due date between the districts.

alpha ($\alpha$): 1 - confidence level (95% confidence level -> $\alpha = .05$)

In [ ]:
get_anova_test(train)

#### Hypothesis Testing #2 (Chi $^2$)

$H_0$: There is no difference in if a case resolution is late between the districts.

$H_a$: There is a difference if a case resolution is late date between the districts.

alpha ($\alpha$): 1 - confidence level (95% confidence level -> $\alpha = .05$)

In [ ]:
get_chi_square(train)